In [1]:
import csv
import re
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn import svm
from sklearn.naive_bayes import GaussianNB, MultinomialNB, BernoulliNB
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
import time
from gensim.models import Word2Vec
from nltk.corpus import stopwords  
import nltk
from nltk.stem.lancaster import LancasterStemmer
import sys
import unicodedata

# Read raw text 

In [2]:
train_raw_filepath =  "./2019S1-proj2-data_dos/train-raw.tsv"
dev_raw_filepath =  "./2019S1-proj2-data_dos/dev-raw.tsv"
test_raw_filepath =  "./2019S1-proj2-data_dos/test-raw.tsv"

In [3]:
# read tsv file into a 2D array
def read_tsv(filepath):
    label, text = [], []
    with open(filepath) as raw:
        reader = csv.reader(raw, delimiter="\t", quoting = csv.QUOTE_NONE)
        for row in reader:
            label.append(row[1])
            text.append(row[-1])
    return text, label

# Feature Engineering

In [4]:
def is_all_zero(instance):
    for attr in instance:
        if float(attr) != 0.0:
            return False
    return True

def remove_all_zero_instance(X, y):
    new_X, new_y = [], []
    for i in range(len(y)):
        instance = X[i]
        if is_all_zero(instance) is False:
            new_X.append(X[i])
            new_y.append(y[i])
    return new_X, new_y

## Text Pre-filtering  

In [5]:
2

## WHL calculating 

In [6]:
class Word_Counter:
    # Initializer / Instance Attributes
    def __init__(self, size):
        self.total_word_count = [0]*size
        self.Melbourne_word_count = [0]*size
        self.Sydney_word_count = [0]*size 
        self.Perth_word_count = [0]*size        
        self.Brisbane_word_count = [0]*size        
class WLH_lists:
    # Initializer / Instance Attributes
    def __init__(self):
        self.Melbourne_WLH_list = []
        self.Sydney_WLH_list = [] 
        self.Perth_WLH_list = []        
        self.Brisbane_WLH_list = []        

In [7]:
def word_frequency_counter(counts, lables):
    # create a list to store count
    num_words = counts.shape[1]
    counter = Word_Counter(num_words)
    
    for i in list(range(len(lables))):
        instance = counts[i, :]
        if (lables[i] == "Melbourne"):
            counter.Melbourne_word_count += instance.toarray().sum(axis=0)
        elif (lables[i] == "Sydney"):
            counter.Sydney_word_count  += instance.toarray().sum(axis=0)
        elif (lables[i] == "Perth"):
            counter.Perth_word_count += instance.toarray().sum(axis=0)
        elif (lables[i] == "Brisbane"):
            counter.Brisbane_word_count += instance.toarray().sum(axis=0)
    
    counter.total_word_count = counter.Melbourne_word_count + \
                                counter.Sydney_word_count +\
                                counter.Perth_word_count +\
                                counter.Brisbane_word_count
    return counter

In [8]:
def calculate_WHL_by_state(word_counter, vectoriser):
    WLH_groupby_state = WLH_lists()
    #  total number of word used for each word counter
    Melbourne_total_word_num = word_counter.Melbourne_word_count.sum()
    Brisbane_total_word_num = word_counter.Brisbane_word_count.sum()
    Sydney_total_word_num = word_counter.Sydney_word_count.sum()
    Perth_total_word_num = word_counter.Perth_word_count.sum()
    all_state_total_word_num = word_counter.total_word_count.sum()

    for i in list(range(len(word_counter.total_word_count))):    
    # for i in list(range(100)):
        curr_word_list = [] # [WHL, word, state]
#         curr_word = vectoriser.get_feature_names()[i]

        curr_word_total_prob = word_counter.total_word_count[i]/all_state_total_word_num

        WHL_Mel = (word_counter.Melbourne_word_count[i]/Melbourne_total_word_num)/curr_word_total_prob
        WHL_Syd = (word_counter.Sydney_word_count[i]/Sydney_total_word_num)/curr_word_total_prob
        WHL_Per = (word_counter.Perth_word_count[i]/Perth_total_word_num)/curr_word_total_prob
        WHL_Bri = (word_counter.Brisbane_word_count[i]/Brisbane_total_word_num)/curr_word_total_prob

        WHL_list = [WHL_Mel, WHL_Bri, WHL_Per, WHL_Syd]
        max_WHL = max(WHL_list)
        
        curr_word_list.append(max_WHL)
        curr_word_list.append(i)
        
        if (WHL_Mel == max_WHL):
            WLH_groupby_state.Melbourne_WLH_list.append(curr_word_list)
        elif (WHL_Bri == max_WHL):
            WLH_groupby_state.Brisbane_WLH_list.append(curr_word_list)
        elif (WHL_Per == max_WHL):
            WLH_groupby_state.Perth_WLH_list.append(curr_word_list) 
        elif (WHL_Syd == max_WHL):
            WLH_groupby_state.Sydney_WLH_list.append(curr_word_list)
            
    sorted(WLH_groupby_state.Melbourne_WLH_list, key=lambda x: x[0], reverse=True)
    sorted(WLH_groupby_state.Sydney_WLH_list, key=lambda x: x[0], reverse=True)
    sorted(WLH_groupby_state.Perth_WLH_list, key=lambda x: x[0], reverse=True)
    sorted(WLH_groupby_state.Brisbane_WLH_list, key=lambda x: x[0], reverse=True)
    return WLH_groupby_state

In [9]:
def groupby_state(WLHs):
    WLH_groupby_state = WLH_lists()
        # Initializer / Instance Attributes

        
    for WLH in WLHs:
        state = WLH[3][0][0]
        if state == "Melbourne":
            WLH_groupby_state.Melbourne_WLH_list.append(WLH)
        elif state == "Sydney":
            WLH_groupby_state.Sydney_WLH_list.append(WLH)
        elif state == "Perth":
            WLH_groupby_state.Perth_WLH_list.append(WLH)            
        elif state == "Brisbane":
            WLH_groupby_state.Brisbane_WLH_list.append(WLH)
    sorted(WLH_groupby_state.Melbourne_WLH_list, key=lambda x: x[0], reverse=True)
    sorted(WLH_groupby_state.Sydney_WLH_list, key=lambda x: x[0], reverse=True)
    sorted(WLH_groupby_state.Perth_WLH_list, key=lambda x: x[0], reverse=True)
    sorted(WLH_groupby_state.Brisbane_WLH_list, key=lambda x: x[0], reverse=True)
    return WLH_groupby_state

In [10]:
def occur_in_n_state(WHL_list):
    count = 0
    for WHL in WHL_list:
        if WHL > 0:
            count += 1
    return count

# Extracting Top k WHL for each state 

In [11]:
def extract_top_k_index(state_WLH_list, k):
    indice = []
    WLH_top_k = state_WLH_list[:k]
    
    for WLH in WLH_top_k:
        indice.append(WLH[1])
    return indice

In [12]:
# select cloumn from X_train by top 10 index for each state
def extract_column_by_index(matrix, indice, vectoriser):
    selected_words = []
    for index in indice:
        selected_words.append(vectoriser.get_feature_names()[index])
    
    result = []
    length = matrix.shape[0]
    for i in list(range(length)):
        row = matrix[i, :].toarray().sum(axis = 0)
        extract = []
        for index in indice:
            extract.append(row[index])
        result.append(extract)
    return result, selected_words
# feature engineering done

## Put all together => Feature Engineering Function

In [13]:
def preprocessing_test(filepath, selected_words):
    # read the file into two parts, text and its location
    X_raw, y = read_tsv(filepath)
    
    # remove URL, hash tag and metion
    remove_URL(X_raw)
#     remove_hash(X_raw)
#     remove_metion(X_raw)
    
    # initilaze vectoriser
    vectoriser = CountVectorizer(stop_words="english", vocabulary=selected_words)
    X_sparse = vectoriser.fit_transform(X_raw)
    
    
    # make sparse matrix into 2D list
    X = []
    for i in list(range(len(y))):
        X.append(X_sparse[i, :].toarray().sum(axis = 0))
    return X, y

In [14]:
#This function takes in 
#@param: filepath => The filepath of the corpus
def preprocessing_train(filepath, k):
    # read the file into two parts, text and its location
    X_raw, y_train = read_tsv(filepath)
    
    # remove URL, hash tag and metion
    remove_URL(X_raw)
    remove_hash(X_raw)
    remove_metion(X_raw)
    
    # initilaze vectoriser
    vectoriser = CountVectorizer(stop_words="english", min_df = 15 )
    X_train = vectoriser.fit_transform(X_raw)
    
    # calculating WLH
    word_counter = word_frequency_counter(X_train, y_train)

    # count frequency for each word by state
    WLH_lists_gourpby_state = calculate_WHL_by_state(word_counter, vectoriser)

    # group WLH by state
#     WLH_lists_gourpby_state = groupby_state(WLH_list_occur_more_than_one_state)
    
    # select top k WHL
#     k = 100
    
    # extract top 10 WLH index for each state
    top_k_indice_for_each_state = []
    top_k_indice_for_each_state.extend(extract_top_k_index(WLH_lists_gourpby_state.Brisbane_WLH_list, k))
    top_k_indice_for_each_state.extend(extract_top_k_index(WLH_lists_gourpby_state.Melbourne_WLH_list, k))
    top_k_indice_for_each_state.extend(extract_top_k_index(WLH_lists_gourpby_state.Perth_WLH_list, k))
    top_k_indice_for_each_state.extend(extract_top_k_index(WLH_lists_gourpby_state.Sydney_WLH_list, k))
    
    # extract columns
    new_train, selected_words = extract_column_by_index(X_train, top_k_indice_for_each_state, vectoriser)
    return new_train, y_train, selected_words

In [15]:
def print_0_percet(X, y):
    num_instance = len(X)
    X_no_zero, y_no_zero = remove_all_zero_instance(X, y)

    num_all_zero_instance = num_instance - len(X_no_zero)
    print("The size of total dataset is %d"%num_instance)
    print("The size of all zero instance in this dataset is %d"%num_all_zero_instance)
    print("The ratio of all zero instances is %f "%(num_all_zero_instance/num_instance))

# Main 

In [20]:
train_small_filepath = "./train_raw_small.csv"
dev_small_filepath = "./dev_raw_small.csv"

In [49]:
X_train, y_train, selected_words = preprocessing_train(train_raw_filepath)

In [22]:
print_0_percet(X_train, y_train)

The size of total dataset is 103364
The size of all zero instance in this dataset is 96990
The ratio of all zero instances is 0.938334 


In [41]:
len(X_train[0])

93

In [54]:
X_dev, y_dev = preprocessing_test(dev_raw_filepath, selected_words)
# print_0_percet(X_dev, y_dev)

In [37]:
X_test, y_test = preprocessing_test(test_raw_filepath, selected_words)
print_0_percet(X_test, y_test)

The size of total dataset is 108148
The size of all zero instance in this dataset is 90986
The ratio of all zero instances is 0.841310 


# Model Training & Evaluation 

In [48]:
models = [DummyClassifier(strategy='most_frequent'),
          GaussianNB(),
          MultinomialNB(),
          BernoulliNB(),
          LogisticRegression()]
titles = ['Zero-R',
          'GNB',
          'MNB',
          'BNB',
          'Logistic Regression']

In [33]:
def put_into_dict(label, dict_):
    if label == "Melbourne":
        dict_["Mel"] += 1
    elif label == "Sydney":
        dict_["Syd"] += 1
    elif label == "Perth":
        dict_["Per"] += 1
    elif label == "Brisbane":
        dict_["Bri"] += 1
        

In [64]:
'''
Predict| Mel |Syd |Per |Bri 
Actual
------------------------
Mel |
Syd |
Per |
Bri |
Tot |
'''
def confusion_matrix_generator(predict_labels, actual_labels):
    # It is a 2D array for store information
    summary_dict = {}
    summary_dict["Mel"] = {"Mel": 0, 'Syd':0, "Per":0, "Bri": 0}
    summary_dict["Syd"] = {"Mel": 0, 'Syd':0, "Per":0, "Bri": 0}
    summary_dict["Per"] = {"Mel": 0, 'Syd':0, "Per":0, "Bri": 0}
    summary_dict["Bri"] = {"Mel": 0, 'Syd':0, "Per":0, "Bri": 0}
    
    for i in list(range(len(actual_labels))):
        label = actual_labels[i]
        prediction = predict_labels[i]
        if label == "Melbourne":
            put_into_dict(prediction, summary_dict["Mel"])
        elif label == "Sydney":
            put_into_dict(prediction, summary_dict["Syd"])
        elif label == "Perth":
            put_into_dict(prediction, summary_dict["Per"])  
        elif label == "Brisbane":
            put_into_dict(prediction, summary_dict["Bri"])          
    
    return summary_dict

In [20]:
k = 10
X_train, y_train, selected_words = preprocessing_train(train_raw_filepath, k)
X_dev, y_dev = preprocessing_test(dev_raw_filepath, selected_words)
model =  LogisticRegression()
model.fit(X_train, y_train)
prediction_dev = model.predict(X_dev)
summary_matrix = confusion_matrix_generator(prediction_dev, y_dev)
print(summary_matrix)

MemoryError: 

In [19]:
prediction_dev = model.predict(X_dev)

NotFittedError: This LogisticRegression instance is not fitted yet

In [59]:
print(prediction_dev)

['Brisbane' 'Brisbane' 'Brisbane' ... 'Brisbane' 'Brisbane' 'Brisbane']


In [65]:
summary_matrix = confusion_matrix_generator(prediction_dev, y_dev)

In [66]:
summary_matrix

{'Mel': {'Mel': 246, 'Syd': 183, 'Per': 134, 'Bri': 8766},
 'Syd': {'Mel': 185, 'Syd': 186, 'Per': 131, 'Bri': 8827},
 'Per': {'Mel': 240, 'Syd': 165, 'Per': 162, 'Bri': 8762},
 'Bri': {'Mel': 202, 'Syd': 176, 'Per': 146, 'Bri': 8805}}

In [51]:
# read csv file
# i = 1
# train_X, train_y = load_dataset(train_filepath[i])
# X_dev_no_0, y_dev_no_0 = remove_all_zero_instance(X_dev, y_dev)
# dev_X, dev_y = load_dataset(dev_filepath[i])


# try each model without feature selection
for title, model in zip(titles, models):
    start = time.time()
    model.fit(X_train, y_train)
    acc = np.mean(cross_val_score(model, X_dev, y_dev, cv=10))
#     acc = np.mean(cross_val_score(model, X_train, y_train, cv=10))
    end = time.time()
    t = end - start
    print(title, acc, 'time:', t)

MemoryError: 

## Stacking 

In [42]:
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.naive_bayes import GaussianNB, MultinomialNB, BernoulliNB
from sklearn.model_selection import cross_val_score
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn import svm
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import time
import time
import numpy as np

np.random.seed(1)

class StackingClassifier():

    def __init__(self, classifiers, metaclassifier):
        self.classifiers = classifiers
        self.metaclassifier = metaclassifier

    def fit(self, X, y):
        for clf in self.classifiers:
            clf.fit(X, y)
        X_meta = self._predict_base(X)
        self.metaclassifier.fit(X_meta, y)
    
    def _predict_base(self, X):
        yhats = []
        for clf in self.classifiers:
            yhat = clf.predict_proba(X)
            yhats.append(yhat)
        yhats = np.concatenate(yhats, axis=1)
        #print(yhats.shape)
        assert len(yhats) == len(X)
        return yhats
    
    def predict(self, X):
        X_meta = self._predict_base(X)
        yhat = self.metaclassifier.predict(X_meta)
        return yhat
    def score(self, X, y):
        yhat = self.predict(X)
        return accuracy_score(y, yhat)
    


classifiers = [LogisticRegression(),
                KNeighborsClassifier(),
                GaussianNB(),
                MultinomialNB(),
                DecisionTreeClassifier()]
              
meta_classifier = DecisionTreeClassifier()
stacker = StackingClassifier(classifiers, meta_classifier)

def load_car_data(car_file):
    X = []
    y = []
    with open(car_file, mode='r') as fin:
        for line in fin:
            atts = line.strip().split(",")
            X.append(atts[:-1]) #all atts minus the last one
            y.append(atts[-1])
    onehot = OneHotEncoder()
    X = onehot.fit_transform(X).toarray()
    return X, y
# X, y = load_car_data('car.data')
# print('labels:', set(y))
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
# stacker.fit(X_train, y_train)
# print('stacker acc:', stacker.score(X_test, y_test))

In [43]:
stacker.fit(X_train, y_train)

In [34]:
print('stacker acc on train:', stacker.score(X_train, y_train))

stacker acc on train: 0.3938895553577648


In [35]:
cv = 5
print('stacker cross-val acc  on train:', np.mean(cross_val_score(stacker.metaclassifier, X_train, y_train, cv=cv)))

stacker cross-val acc  on train: 0.37537234507695616


In [36]:
print('stacker acc on dev:', stacker.score(X_dev,y_dev))

stacker acc on dev: 0.31570907921534996


In [24]:
model = KNeighborsClassifier(n_neighbors=3, algorithm='auto', metric = "cosine")
model.fit(X_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='cosine',
           metric_params=None, n_jobs=1, n_neighbors=3, p=2,
           weights='uniform')

In [25]:
np.mean(cross_val_score(model, X_dev, y_dev, cv=10))

0.2890342420861846

In [51]:
model =  LogisticRegression()
model.fit(X_train, y_train)

C:\Users\91260\AppData\Local\Programs\Python\Python36-32\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\91260\AppData\Local\Programs\Python\Python36-32\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [64]:
model = MultinomialNB()
model.fit(X_train, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [65]:
predict_labels = model.predict(X_test)

In [ ]:
predict_labels = stacker.predict(X_test)

In [ ]:
def test_prediction_to_submit_file(test_lables, filepath):
    with open(filepath, 'w',  newline='') as writeFile:
        writer = csv.writer(writeFile)
        writer.writerows([["Id", "Class"]])
        for i in list(range(len(test_lables))):
            index = '3' + str(i+1)
            writer.writerows([[index, test_lables[i]]])
        writeFile.close()

In [ ]:
test_prediction_to_submit_file(predict_labels, "predicted_labels.csv")